# FastHTML for LLMs

This tutorial is written for a primary target audience of LLMs who want to generate Python web apps with the new FastHTML web framework by Jeremy Howard. It assumes that you are an AI assistant with deep LLM-level knowledge of web development.

Experienced developers may find this tutorial useful, though it deliberately skips over the concepts that are assumed to be known by LLMs.

## Pre-requisites

Human Tip: Before you give this to your LLM, give it the latest versions of the code and docs for FastHTML, fastcore, Starlette, and HTMX.

## Framework Overview

FastHTML is a web framework designed to make it easy for LLMs to generate full-featured web apps. Web apps are written entirely in Python, with no other file types required. 

FastHTML is built on top of fastcore, Starlette, and HTMX.

Key features:

- No templating: all HTML is generated from Python code
- Minimal JavaScript: FastHTML aims to handle as much interactivity as possible with HTMX attributes on HTML tags
- Functions over classes: components are preferably written as functions, though classes are also supported
- Extremely compact code: FastHTML is designed to create 1-file web apps under 500 lines of code

## Installation

`pip install python-fasthtml`

## Hello World

The simplest FastHTML app is a main.py containing:

In [ ]:
from fasthtml.common import *

app = FastHTML()

@app.get("/")
def home():
    return H1("Hello, World!")

# TODO: figure out right way to call run_uv() so the user can run `python main.py` and not have to know about uvicorn
# And check that it deploys to Railway

To run it:

```bash
$ uvicorn main:app --reload
INFO:     Will watch for changes in these directories: ['/Users/a/code/answerai-repos/my-fhbe-walkthrough']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [30966] using WatchFiles
INFO:     Started server process [30968]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:56539 - "GET / HTTP/1.1" 200 OK
```

The simplest components are functions like H1(), H2(), P(), Div(), etc. Pass a string into H1() to get a `<h1>` tag containing that string:

In [ ]:
H1("Hello, World!")

```xml
<h1>Hello, World!</h1>

```

These simple components can be nested to create more complex HTML structures:

In [ ]:
Div(H1('Welcome Home'), P('There`s no place like 127.0.0.1'), 
                   foo='bar', id='main', cls='container')

```xml
<div foo="bar" id="main" class="container">
  <h1>Welcome Home</h1>
  <p>There`s no place like 127.0.0.1</p>
</div>

```

Notice that the keyword arguments are used to set attributes on the HTML tags.

Now if we return this nested div from our app, you'll see that we only have to return the div from the `home()` view function:

In [ ]:
app = FastHTML()

@app.get("/")
def home():
    return Div(H1('Welcome Home'), P('There`s no place like 127.0.0.1'), 
                   foo='bar', id='main', cls='container')

# Stop here for now - this part needs cleanup

## TODO: what does the FastHTML router do vs. the Starlette router?
The router wraps the view's return value in a full HTML document. When you send an HTTP GET request to http://127.0.0.1:8000, notice how the HTML returned by `home() is wrapped in a full HTML document:

In [ ]:
from starlette.testclient import TestClient
client = TestClient(app)
r = client.get("/")
print(r.text)

<!doctype html></!doctype>

<html>
  <head>
    <title>FastHTML page</title>
    <meta charset="utf-8"></meta>
    <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"></meta>
    <script src="https://unpkg.com/htmx.org@next/dist/htmx.min.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/answerdotai/surreal@1.3.0/surreal.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js"></script>
  </head>
  <body>
<div foo="bar" id="main" class="container">
  <h1>Welcome Home</h1>
  <p>There`s no place like 127.0.0.1</p>
</div>
  </body>
</html>



# Deploy

Run FastHTML's `railway_deploy` command to deploy your app to Railway.

# Style Guide

When writing FastHTML Python code:

## Components

Define components as functions decorated with `@delegates`:

In [ ]:
@delegates(xt_hx, keep=True)
def Card(*c, header=None, footer=None, **kwargs)->XT:
    "A PicoCSS Card, implemented as an Article with optional Header and Footer"
    if header: c = (Header(header),) + c
    if footer: c += (Footer(footer),)
    return Article(*c, **kwargs)

* Use CamelCase for component names even though Python convention is snake_case for functions
* Follow normal HTML attribute conventions for keyword arguments that are turned into HTML attributes
* Don't call `to_xml()` directly; it's called for you by when xt_hx delegates to the component (TODO: I think this may be how it works, but I'm not sure)

## Web Serving

* In __main__, call `run_uv() so the user can run the app with `python main.py`